In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
X_train = pd.read_csv('X_train_ver2.csv')
X_test = pd.read_csv('X_test_ver2.csv')
y_train = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_label.csv', engine = 'python')

In [3]:
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [6]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [7]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [8]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [9]:
train = pd.merge(X_train, y_train, how = 'left', on = 'acc_id')
test = pd.merge(X_test, y_test, how = 'left', on = 'acc_id')

In [10]:
train_leave = train[train['leave'] == 1]
train_amount = train[train['amount'] == 1]

## 추가 생존 기간

In [11]:
train_leave.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell,survival_time,amount_spent,leave,amount
1,5.0,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,...,0,0,0,0,0,0,60,0.000000,1,0
5,31.0,17,19.586603,22.231543,21.236765,19.027186,15.761864,13.541598,18.448150,17.846955,...,0,1,0,1,0,1,45,0.051316,1,1
7,41.0,0,0.000000,0.000000,0.000000,0.004681,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,3,0.000000,1,0
11,54.0,11,2.359380,8.381886,8.868743,11.019805,0.650535,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,50,0.059060,1,1
13,63.0,17,5.435001,15.591104,5.989737,8.611271,0.000000,0.000000,0.026697,0.809029,...,0,1,0,0,0,0,22,0.067853,1,1


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   45.0s remaining:   51.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [12, 13, 14, 15],


In [13]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-204.21454773835353
{'max_features': 14, 'n_estimators': 750}


In [16]:
from sklearn.externals import joblib

rf_leave_reg = grid_search.best_estimator_
print(rf_leave_reg)
joblib.dump(rf_leave_reg, './빅콘테스트/model/version2/rf_leave_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=14, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=750,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_leave_reg_version2.pkl']

In [17]:
Importance = pd.DataFrame({'feature' : train_leave.columns[1:-4], 'importance' : rf_leave_reg.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
49,enchant_count_1,0.000031
50,enchant_count_2,0.000042
52,enchant_count_4,0.000112
51,enchant_count_3,0.000181
108,source_type_0_week3_enchant_scroll,0.000373
106,source_type_0_week1_enchant_scroll,0.000407
85,server_group_2,0.000419
110,source_type_1_week1_enchant_scroll,0.000429
112,source_type_1_week3_enchant_scroll,0.000459
83,server_group_0,0.000467


In [18]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_jobs', refit=True,

In [19]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-211.16980140049122
{'gamma': 1, 'learning_rate': 0.1, 'max_depth': 9}


In [20]:
xgb_leave_reg = grid_search.best_estimator_
print(xgb_leave_reg)
joblib.dump(xgb_leave_reg, './빅콘테스트/model/version2/xgb_leave_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_leave_reg_version2.pkl']

In [21]:
Importance = pd.DataFrame({'feature' : train_leave.columns[1:-4], 'importance' : xgb_leave_reg.feature_importances_}).sort_values(by = 'importance', ascending = True)
Importance

,feature,importance
50,enchant_count_2,0.000000
51,enchant_count_3,0.000000
49,enchant_count_1,0.000055
108,source_type_0_week3_enchant_scroll,0.000055
52,enchant_count_4,0.000109
85,server_group_2,0.000219
84,server_group_1,0.000219
70,combat_same_pledge_cnt_2,0.000328
107,source_type_0_week2_enchant_scroll,0.000328
106,source_type_0_week1_enchant_scroll,0.000328


In [22]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_leave_reg = VotingRegressor(
    estimators=[('rf', rf_leave_reg), ('xgb', xgb_leave_reg)])

cv_value = cross_val_score(voting_leave_reg, train_leave.iloc[:, 1:-4],
                           train_leave['survival_time'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-204.19417144497024

In [23]:
voting_leave_reg.fit(train_leave.iloc[:, 1:-4], train_leave['survival_time'])
print(voting_leave_reg)
joblib.dump(voting_leave_reg, './빅콘테스트/model/version2/voting_leave_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=14,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=750,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_leave_reg_version2.pkl']

## 일 평균 결제 금액

In [24]:
train_amount.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell,survival_time,amount_spent,leave,amount
2,8.0,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,...,0,0,0,0,1,0,64,0.020310,0,1
3,17.0,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,...,0,1,1,1,0,1,64,0.070642,0,1
4,21.0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,...,0,1,0,1,0,0,64,0.184267,0,1
5,31.0,17,19.586603,22.231543,21.236765,19.027186,15.761864,13.541598,18.448150,17.846955,...,0,1,0,1,0,1,45,0.051316,1,1
6,38.0,17,21.463808,21.093985,11.897550,16.354158,49.506198,13.210755,8.478579,11.016509,...,1,0,0,0,0,0,64,0.517168,0,1


In [25]:
rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.9min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [12, 13, 14, 15],


In [26]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-0.7281713953912863
{'max_features': 13, 'n_estimators': 300}


In [27]:
rf_amount_reg = grid_search.best_estimator_
print(rf_amount_reg)
joblib.dump(rf_amount_reg, './빅콘테스트/model/version2/rf_amount_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=13, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_amount_reg_version2.pkl']

In [28]:
xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_amount.iloc[:, 1:-4], train_amount['amount_spent'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  2.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_jobs', refit=True,

In [29]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-0.8351116045072274
{'gamma': 1, 'learning_rate': 0.01, 'max_depth': 9}


In [30]:
xgb_amount_reg = grid_search.best_estimator_
print(xgb_amount_reg)
joblib.dump(xgb_amount_reg, './빅콘테스트/model/version2/xgb_amount_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_amount_reg_version2.pkl']

In [31]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_amount_reg = VotingRegressor(
    estimators=[('rf', rf_amount_reg), ('xgb', xgb_amount_reg)])

cv_value = cross_val_score(voting_amount_reg, train_amount.iloc[:, 1:-4],
                           train_amount['amount_spent'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-0.7603123146247154

In [32]:
voting_amount_reg.fit(train_amount.iloc[:, 1:-4], train_amount['survival_time'])
print(voting_amount_reg)
joblib.dump(voting_amount_reg, './빅콘테스트/model/version2/voting_amount_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=13,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=300,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_amount_reg_version2.pkl']

## 일 평균 결제 금액(all data)

In [11]:
train.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell,survival_time,amount_spent,leave,amount
0,2.0,0,22.919696,22.980553,22.659883,22.973531,0.000000,0.000000,0.000000,0.000000,...,1,0,0,0,0,0,64,0.000000,0,0
1,5.0,15,0.000000,0.589845,0.213000,5.313287,0.000000,0.310229,0.162887,2.763336,...,0,0,0,0,0,0,60,0.000000,1,0
2,8.0,16,4.987936,4.889629,4.653223,4.267014,26.808113,1.325063,1.444694,1.594739,...,0,0,0,0,1,0,64,0.020310,0,1
3,17.0,16,5.406914,9.048973,7.831832,6.176989,25.931158,1.328780,1.362575,1.834339,...,0,1,1,1,0,1,64,0.070642,0,1
4,21.0,16,6.048253,7.363701,4.966870,3.452466,4.379366,4.602406,0.654252,1.135141,...,0,1,0,1,0,0,64,0.184267,0,1


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [10, 11, 12, 13]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train.iloc[:, 1:-4], train['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  1.8min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [10, 11, 12, 13],


In [20]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-0.4656225210644694
{'max_features': 12, 'n_estimators': 300}


In [21]:
from sklearn.externals import joblib

rf_amount_all_reg = grid_search.best_estimator_
print(rf_amount_all_reg)
joblib.dump(rf_amount_all_reg, './빅콘테스트/model/version2/rf_amount_all_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=12, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_amount_all_reg_version2.pkl']

In [22]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [7, 8, 9, 10], 'gamma' : [0, 1, 3, 5]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train.iloc[:, 1:-4], train['amount_spent'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  3.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3, 5],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [7, 8, 9, 10]}],
             pre_dispatch='2*n_jobs', refit=True

In [23]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-0.5110501074116188
{'gamma': 3, 'learning_rate': 0.01, 'max_depth': 8}


In [24]:
from sklearn.externals import joblib

xgb_amount_all_reg = grid_search.best_estimator_
print(xgb_amount_all_reg)
joblib.dump(xgb_amount_all_reg, './빅콘테스트/model/version2/xgb_amount_all_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=3, learning_rate=0.01, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_amount_all_reg_version2.pkl']

In [25]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_amount_all_reg = VotingRegressor(
    estimators=[('rf', rf_amount_all_reg), ('xgb', xgb_amount_all_reg)])

cv_value = cross_val_score(voting_amount_all_reg, train.iloc[:, 1:-4],
                           train['amount_spent'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-0.4727506325806855

In [26]:
voting_amount_all_reg.fit(train.iloc[:, 1:-4], train['survival_time'])
print(voting_amount_all_reg)
joblib.dump(voting_amount_all_reg, './빅콘테스트/model/version2/voting_amount_all_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=12,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=300,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_amount_all_reg_version2.pkl']

## 일 평균 결제 금액(이탈 & 결제)

In [27]:
train_sub = train[(train['leave'] == 1) & (train['amount'] == 1)]

In [29]:
train_sub.head()

,acc_id,level,playtime_1,playtime_2,playtime_3,playtime_4,npc_kill_1,npc_kill_2,npc_kill_3,npc_kill_4,...,source_armor,target_armor,source_accessory,target_accessory,source_spell,target_spell,survival_time,amount_spent,leave,amount
5,31.0,17,19.586603,22.231543,21.236765,19.027186,15.761864,13.541598,18.448150,17.846955,...,0,1,0,1,0,1,45,0.051316,1,1
11,54.0,11,2.359380,8.381886,8.868743,11.019805,0.650535,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,50,0.059060,1,1
13,63.0,17,5.435001,15.591104,5.989737,8.611271,0.000000,0.000000,0.026697,0.809029,...,0,1,0,0,0,0,22,0.067853,1,1
15,66.0,16,10.018004,8.644040,12.496758,15.420237,19.709681,0.694805,3.805544,4.134021,...,1,1,0,1,0,0,42,0.033763,1,1
34,154.0,15,4.880266,2.080842,3.527367,1.888909,6.550969,0.431887,0.924266,1.148658,...,0,0,1,1,1,1,2,0.049217,1,1


In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [10, 11, 12, 13]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_sub.iloc[:, 1:-4], train_sub['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   24.3s remaining:   27.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   43.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [10, 11, 12, 13],


In [31]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9012098216383093
{'max_features': 11, 'n_estimators': 500}


In [32]:
from sklearn.externals import joblib

rf_amount_sub_reg = grid_search.best_estimator_
print(rf_amount_sub_reg)
joblib.dump(rf_amount_sub_reg, './빅콘테스트/model/version2/rf_amount_sub_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=11, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_amount_sub_reg_version2.pkl']

In [41]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [1, 2, 3, 4], 'gamma' : [0, 1, 3, 10]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_sub.iloc[:, 1:-4], train_sub['amount_spent'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   17.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3, 10],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [1, 2, 3, 4]}],
             pre_dispatch='2*n_jobs', refit=True

In [42]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9869484252715186
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 1}


In [43]:
from sklearn.externals import joblib

xgb_amount_sub_reg = grid_search.best_estimator_
print(xgb_amount_sub_reg)
joblib.dump(xgb_amount_sub_reg, './빅콘테스트/model/version2/xgb_amount_sub_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
             max_depth=1, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_amount_sub_reg_version2.pkl']

In [44]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_amount_sub_reg = VotingRegressor(
    estimators=[('rf', rf_amount_sub_reg), ('xgb', xgb_amount_sub_reg)])

cv_value = cross_val_score(voting_amount_sub_reg, train_sub.iloc[:, 1:-4],
                           train_sub['amount_spent'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-1.9064826612472736

In [45]:
voting_amount_sub_reg.fit(train_sub.iloc[:, 1:-4], train_sub['survival_time'])
print(voting_amount_sub_reg)
joblib.dump(voting_amount_sub_reg, './빅콘테스트/model/version2/voting_amount_sub_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=11,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=500,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_amount_sub_reg_version2.pkl']

# non class

In [47]:
X_train_non_class = pd.read_csv('X_train_ver2_non_class.csv')
X_test_non_class = pd.read_csv('X_test_ver2_non_class.csv')
y_train = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_label.csv', engine = 'python')

In [48]:
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [51]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [52]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [53]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [54]:
train_non_class = pd.merge(X_train_non_class, y_train, how = 'left', on = 'acc_id')
test_non_class = pd.merge(X_test_non_class, y_test, how = 'left', on = 'acc_id')

In [55]:
train_non_class_leave = train_non_class[train_non_class['leave'] == 1]
train_non_class_sub = train_non_class[(train_non_class['leave'] == 1) & (train_non_class['amount'] == 1)]

## 추가 생존 기간

### RandomForest

In [56]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_non_class_leave.iloc[:, 1:-4], train_non_class_leave['survival_time'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   46.1s remaining:   52.7s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [12, 13, 14, 15],


In [57]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-204.25867123026995
{'max_features': 14, 'n_estimators': 750}


In [58]:
from sklearn.externals import joblib

rf_leave_non_class_reg = grid_search.best_estimator_
print(rf_leave_non_class_reg)
joblib.dump(rf_leave_non_class_reg, './빅콘테스트/model/version2/rf_leave_non_class_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=14, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=750,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_leave_non_class_reg_version2.pkl']

### XGBoost

In [59]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_non_class_leave.iloc[:, 1:-4], train_non_class_leave['survival_time'])

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_jobs', refit=True,

In [60]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-212.0057736319297
{'gamma': 1, 'learning_rate': 0.1, 'max_depth': 9}


In [61]:
xgb_leave_non_class_reg = grid_search.best_estimator_
print(xgb_leave_non_class_reg)
joblib.dump(xgb_leave_non_class_reg, './빅콘테스트/model/version2/xgb_leave_non_class_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_leave_non_class_reg_version2.pkl']

### Ensemble

In [62]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_leave_non_class_reg = VotingRegressor(
    estimators=[('rf', rf_leave_non_class_reg), ('xgb', xgb_leave_non_class_reg)])

cv_value = cross_val_score(voting_leave_non_class_reg, train_non_class_leave.iloc[:, 1:-4],
                           train_non_class_leave['survival_time'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-204.61099470835364

In [63]:
voting_leave_non_class_reg.fit(train_non_class_leave.iloc[:, 1:-4], train_non_class_leave['survival_time'])
print(voting_leave_non_class_reg)
joblib.dump(voting_leave_non_class_reg, './빅콘테스트/model/version2/voting_leave_non_class_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=14,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=750,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_leave_non_class_reg_version2.pkl']

## 일 평균 결제 금액

### RandomForest

In [65]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [10, 11, 12, 13]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_non_class_sub.iloc[:, 1:-4], train_non_class_sub['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   25.8s remaining:   29.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   45.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [10, 11, 12, 13],


In [66]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9002387579085536
{'max_features': 10, 'n_estimators': 300}


In [67]:
from sklearn.externals import joblib

rf_amount_non_class_sub_reg = grid_search.best_estimator_
print(rf_amount_non_class_sub_reg)
joblib.dump(rf_amount_non_class_sub_reg, './빅콘테스트/model/version2/rf_amount_non_class_sub_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=10, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_amount_non_class_sub_reg_version2.pkl']

### XGBoost

In [68]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [1, 2, 3, 4], 'gamma' : [0, 1, 3, 10]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_non_class_sub.iloc[:, 1:-4], train_non_class_sub['amount_spent'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   17.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3, 10],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [1, 2, 3, 4]}],
             pre_dispatch='2*n_jobs', refit=True

In [69]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9869484252715186
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 1}


In [70]:
from sklearn.externals import joblib

xgb_amount_non_class_sub_reg = grid_search.best_estimator_
print(xgb_amount_non_class_sub_reg)
joblib.dump(xgb_amount_non_class_sub_reg, './빅콘테스트/model/version2/xgb_amount_non_class_sub_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
             max_depth=1, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_amount_non_class_sub_reg_version2.pkl']

### Ensemble

In [71]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_amount_non_class_sub_reg = VotingRegressor(
    estimators=[('rf', rf_amount_non_class_sub_reg), ('xgb', xgb_amount_non_class_sub_reg)])

cv_value = cross_val_score(voting_amount_non_class_sub_reg, train_non_class_sub.iloc[:, 1:-4],
                           train_non_class_sub['amount_spent'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-1.9017403430575317

In [72]:
voting_amount_non_class_sub_reg.fit(train_non_class_sub.iloc[:, 1:-4], train_non_class_sub['survival_time'])
print(voting_amount_non_class_sub_reg)
joblib.dump(voting_amount_non_class_sub_reg, './빅콘테스트/model/version2/voting_amount_non_class_sub_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=10,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=300,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_amount_non_class_sub_reg_version2.pkl']

# 플레이 시간 변동

In [510]:
X_train_fluctuation = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test_fluctuation = pd.read_csv('X_test_ver2_fluctuation.csv')
y_train = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_label.csv', engine = 'python')

In [511]:
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [512]:
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [513]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [514]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [515]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [516]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [517]:
train_fluctuation = pd.merge(X_train_fluctuation, y_train, how = 'left', on = 'acc_id')
test_fluctuation = pd.merge(X_test_fluctuation, y_test, how = 'left', on = 'acc_id')

In [518]:
train_fluctuation_leave = train_fluctuation[train_fluctuation['leave'] == 1]
train_fluctuation_sub = train_fluctuation[(train_fluctuation['leave'] == 1) & (train_fluctuation['amount'] == 1)]

## 추가 생존 기간

### RandomForest

In [519]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

# 12, 13, 14, 15
param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:  2.0min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  3.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [12, 13, 14, 15],


In [520]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-202.65353874334156
{'max_features': 14, 'n_estimators': 750}


In [521]:
from sklearn.externals import joblib

# rf_leave_fluctuation_reg = RandomForestClassifier(random_state = 42, max_features = 14, n_estimators = 750)
# rf_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])
rf_leave_fluctuation_reg = grid_search.best_estimator_
print(rf_leave_fluctuation_reg)
joblib.dump(rf_leave_fluctuation_reg, './빅콘테스트/model/version2/rf_leave_fluctuation_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=14, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=750,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_leave_fluctuation_reg_version2.pkl']

### XGBoost

In [522]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

# 9, 10, 11, 12
param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [9, 10, 11, 12], 'gamma' : [0, 1, 3, 5]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3, 5],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [9, 10, 11, 12]}],
             pre_dispatch='2*n_jobs', refit=Tr

In [523]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-211.56103242840211
{'gamma': 0, 'learning_rate': 0.1, 'max_depth': 9}


In [524]:
# xgb_leave_fluctuation_reg = XGBClassifier(random_state = 42, learning_rate = 0.1, gamma = 0, max_depth = 9)
# xgb_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])

xgb_leave_fluctuation_reg = grid_search.best_estimator_
print(xgb_leave_fluctuation_reg)
joblib.dump(xgb_leave_fluctuation_reg, './빅콘테스트/model/version2/xgb_leave_fluctuation_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_leave_fluctuation_reg_version2.pkl']

### Ensemble

In [525]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_leave_fluctuation_reg = VotingRegressor(
    estimators=[('rf', rf_leave_fluctuation_reg), ('xgb', xgb_leave_fluctuation_reg)])

cv_value = cross_val_score(voting_leave_fluctuation_reg, train_fluctuation_leave.iloc[:, 1:-4],
                           train_fluctuation_leave['survival_time'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-203.67826900395812

In [526]:
voting_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])
print(voting_leave_fluctuation_reg)
joblib.dump(voting_leave_fluctuation_reg, './빅콘테스트/model/version2/voting_leave_fluctuation_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=14,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=750,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                

['./빅콘테스트/model/version2/voting_leave_fluctuation_reg_version2.pkl']

## 일 평균 결제 금액

### RandomForest

In [527]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [8, 9, 10, 11]}
  ]

grid_search = GridSearchCV(rf, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   35.3s remaining:   40.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   54.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [8, 9, 10, 11],
  

In [528]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9092445241882556
{'max_features': 9, 'n_estimators': 300}


In [529]:
from sklearn.externals import joblib

# rf_amount_fluctuation_sub_reg = RandomForestClassifier(random_state = 42, max_features = 9, n_estimators = 300)
# rf_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
rf_amount_fluctuation_sub_reg = grid_search.best_estimator_
print(rf_amount_fluctuation_sub_reg)
joblib.dump(rf_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_version2.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=9, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)


['./빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_version2.pkl']

### XGBoost

In [530]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)

param_grid = [
    {'learning_rate' : [0.001, 0.01, 0.1], 'max_depth' : [1, 2, 3, 4], 'gamma' : [0, 1, 3, 10]}
  ]

grid_search = GridSearchCV(xgb, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   16.5s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=42,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1),
             iid='warn', n_jobs=-1,
             param_grid=[{'gamma': [0, 1, 3, 10],
                          'learning_rate': [0.001, 0.01, 0.1],
                          'max_depth': [1, 2, 3, 4]}],
             pre_dispatch='2*n_jobs', refit=True

In [531]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9869484252715186
{'gamma': 0, 'learning_rate': 0.01, 'max_depth': 1}


In [532]:
from sklearn.externals import joblib

# xgb_amount_fluctuation_sub_reg = XGBClassifier(random_state = 42, learning_rate = 0.01, gamma = 0, max_depth = 1)
# xgb_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
xgb_amount_fluctuation_sub_reg = grid_search.best_estimator_
print(xgb_amount_fluctuation_sub_reg)
joblib.dump(xgb_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/xgb_amount_fluctuation_sub_reg_version2.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
             max_depth=1, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)


['./빅콘테스트/model/version2/xgb_amount_fluctuation_sub_reg_version2.pkl']

### Ensemble

In [533]:
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score

voting_amount_fluctuation_sub_reg = VotingRegressor(
    estimators=[('rf', rf_amount_fluctuation_sub_reg), ('xgb', xgb_amount_fluctuation_sub_reg)])

cv_value = cross_val_score(voting_amount_fluctuation_sub_reg, train_fluctuation_sub.iloc[:, 1:-4],
                           train_fluctuation_sub['amount_spent'], cv = 5,
                           scoring = "neg_mean_squared_error")

np.mean(cv_value)

-1.9063391581715519

In [534]:
voting_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
print(voting_amount_fluctuation_sub_reg)
joblib.dump(voting_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/voting_amount_fluctuation_sub_reg_version2.pkl')

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=9,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=300,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=42, verbose=0,
                 

['./빅콘테스트/model/version2/voting_amount_fluctuation_sub_reg_version2.pkl']

# 교차검증 X

In [559]:
X_train_fluctuation = pd.read_csv('X_train_ver2_fluctuation.csv')
X_test_fluctuation = pd.read_csv('X_test_ver2_fluctuation.csv')
y_train = pd.read_csv('/home/user3/anaconda3/빅콘테스트/train_label.csv', engine = 'python')

In [560]:
y_train['leave'] = 0
y_train['leave'][y_train.survival_time < 64] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [561]:
y_train['amount'] = 0
y_train['amount'][y_train.amount_spent > 0] = 1

/home/user3/.conda/envs/hun/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [562]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 42)
for train_index, test_index in split.split(y_train, y_train['leave']):
    strat_y_train = y_train.loc[train_index]
    strat_y_test = y_train.loc[test_index]

In [563]:
len(strat_y_train), len(strat_y_test)

(30000, 10000)

In [564]:
strat_y_train['leave'].mean(), strat_y_test['leave'].mean()

(0.4501, 0.4501)

In [596]:
y_train = strat_y_train.sort_values(['acc_id'], ascending = True)
y_test = strat_y_test.sort_values(['acc_id'], ascending = True)

In [597]:
train_fluctuation = pd.merge(X_train_fluctuation, y_train, how = 'left', on = 'acc_id')
test_fluctuation = pd.merge(X_test_fluctuation, y_test, how = 'left', on = 'acc_id')

In [598]:
train_fluctuation_leave = train_fluctuation[train_fluctuation['leave'] == 1]
train_fluctuation_sub = train_fluctuation[(train_fluctuation['leave'] == 1) & (train_fluctuation['amount'] == 1)]

## 추가 생존 기간

In [568]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

rf_leave_fluctuation_reg = RandomForestRegressor(random_state = 42, max_features = 14, n_estimators = 750)
rf_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])
joblib.dump(rf_leave_fluctuation_reg, './빅콘테스트/model/version2/rf_leave_fluctuation_reg_version2.pkl')

['./빅콘테스트/model/version2/rf_leave_fluctuation_reg_version2.pkl']

In [664]:
from xgboost import XGBRegressor
from sklearn.externals import joblib

xgb_leave_fluctuation_reg = XGBRegressor(random_state = 42, learning_rate = 0.1, gamma = 0, max_depth = 9)
xgb_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])
joblib.dump(xgb_leave_fluctuation_reg, './빅콘테스트/model/version2/xgb_leave_fluctuation_reg_version2.pkl')

['./빅콘테스트/model/version2/xgb_leave_fluctuation_reg_version2.pkl']

In [665]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

extree = ExtraTreesRegressor(random_state = 42, bootstrap = True)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [12, 13, 14, 15]}
  ]

grid_search = GridSearchCV(extree, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   51.6s remaining:   58.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse',
                                           max_depth=None, max_features='auto',
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators='warn', n_jobs=None,
                                           oob_score=False, random_state=42,
                                           verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [12, 13, 14, 15],
                          'n_estimators': [300, 500, 750]}],
      

In [666]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-206.56077085587813
{'max_features': 15, 'n_estimators': 300}


In [667]:
from sklearn.externals import joblib

extree_leave_fluctuation_reg = ExtraTreesRegressor(random_state = 42, bootstrap = True, max_features = 8, n_estimators = 500)
extree_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])
joblib.dump(extree_leave_fluctuation_reg, './빅콘테스트/model/version2/extree_leave_fluctuation_reg_version2.pkl')

['./빅콘테스트/model/version2/extree_leave_fluctuation_reg_version2.pkl']

In [668]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

lr = ElasticNet(random_state = 42)

param_grid = [
    {'alpha' : [0.1, 1, 3, 5], 'l1_ratio' : [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)
grid_search.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    2.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=42, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'alpha': [0.1, 1, 3, 5],
                          'l1_ratio': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [669]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-244.2091386898045
{'alpha': 0.1, 'l1_ratio': 0.6}


In [670]:
from sklearn.externals import joblib

lr_leave_fluctuation_reg = ElasticNet(random_state = 42, alpha = 0.1, l1_ratio = 0.6)
lr_leave_fluctuation_reg.fit(train_fluctuation_leave.iloc[:, 1:-4], train_fluctuation_leave['survival_time'])
joblib.dump(lr_leave_fluctuation_reg, './빅콘테스트/model/version2/lr_leave_fluctuation_reg_version2.pkl')

['./빅콘테스트/model/version2/lr_leave_fluctuation_reg_version2.pkl']

## 일 평균 결제 금액

In [599]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

rf_amount_fluctuation_sub_reg = RandomForestRegressor(random_state = 42, max_features = 9, n_estimators = 300)
rf_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
joblib.dump(rf_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_version2.pkl')

['./빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_version2.pkl']

In [616]:
from xgboost import XGBRegressor
from sklearn.externals import joblib

xgb_amount_fluctuation_sub_reg = XGBRegressor(random_state = 42, learning_rate = 0.01, gamma = 0, max_depth = 1)
xgb_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
joblib.dump(xgb_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/xgb_amount_fluctuation_sub_reg_version2.pkl')

['./빅콘테스트/model/version2/xgb_amount_fluctuation_sub_reg_version2.pkl']

In [619]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV

extree = ExtraTreesRegressor(random_state = 42, bootstrap = True)

param_grid = [
    {'n_estimators' : [300, 500, 750], 'max_features' : [8, 9, 10, 11]}
  ]

grid_search = GridSearchCV(extree, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)

grid_search.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 out of  60 | elapsed:   25.3s remaining:   28.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   32.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse',
                                           max_depth=None, max_features='auto',
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators='warn', n_jobs=None,
                                           oob_score=False, random_state=42,
                                           verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [8, 9, 10, 11],
                          'n_estimators': [300, 500, 750]}],
        

In [620]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-1.9346072029486734
{'max_features': 8, 'n_estimators': 500}


In [621]:
from sklearn.externals import joblib

extree_amount_fluctuation_sub_reg = ExtraTreesRegressor(random_state = 42, bootstrap = True, max_features = 8, n_estimators = 500)
extree_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
joblib.dump(extree_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/extree_amount_fluctuation_sub_reg_version2.pkl')

['./빅콘테스트/model/version2/extree_amount_fluctuation_sub_reg_version2.pkl']

In [622]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

lr = ElasticNet(random_state = 42)

param_grid = [
    {'alpha' : [0.1, 1, 3, 5], 'l1_ratio' : [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
  ]

grid_search = GridSearchCV(lr, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error', verbose = 2, n_jobs = -1)
grid_search.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    0.7s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=42, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'alpha': [0.1, 1, 3, 5],
                          'l1_ratio': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=2)

In [623]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-2.032606803135335
{'alpha': 0.1, 'l1_ratio': 0.4}


In [624]:
from sklearn.externals import joblib

lr_amount_fluctuation_sub_reg = ElasticNet(random_state = 42, alpha = 0.1, l1_ratio = 0.4)
lr_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
joblib.dump(lr_amount_fluctuation_sub_reg, './빅콘테스트/model/version2/lr_amount_fluctuation_sub_reg_version2.pkl')

['./빅콘테스트/model/version2/lr_amount_fluctuation_sub_reg_version2.pkl']

In [627]:
def predict(model1, model2, model3, model4, model5, model6, model7, data, w1, w2, w3, w4):
    '''
    model1 : 생존 여부 예측(classifier)
    model2 : 결제 여부 예측(classifier)
    model3 : 추가 생존 기간 예측(regressor)
    model4~7 : 일 평균 결제 금액 예측(regressor)
    data : test data
    '''
    
    pred1 = model1.predict(data.iloc[:, 1:])
    pred2 = model2.predict(data.iloc[:, 1:])
    pred3 = model3.predict(data.iloc[:, 1:])
    pred4 = ((w1 * model4.predict(data.iloc[:, 1:])) + (w2 * model5.predict(data.iloc[:, 1:]))
            + (w3 * model6.predict(data.iloc[:, 1:])) + (w4 * model7.predict(data.iloc[:, 1:])))
    
    pred3[pred1 == 0] = 64
    pred4[pred2 == 0] = 0
    pred3[pred3 <= 1] = 1
    pred4[pred4 <= 0] = 0
    
    array = np.concatenate([data.iloc[:, 0].values.reshape(-1, 1), pred3.reshape(-1, 1), pred4.reshape(-1, 1)], axis = 1)
    df = pd.DataFrame(array)
    df.columns = ['acc_id', 'survival_time', 'amount_spent']
    df['acc_id'] = df['acc_id'].astype('int32')
    df['survival_time'] = round(df['survival_time']).astype('int32')
    
    return df

In [628]:
X_test_fluctuation = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1_fluctuation = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2_fluctuation = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [629]:
voting_leave_fluctuation_clf_2 = joblib.load('./빅콘테스트/model/version2/voting_leave_fluctuation_clf_2_version2.pkl')
voting_amount_fluctuation_clf_2 = joblib.load('./빅콘테스트/model/version2/voting_amount_fluctuation_clf_2_version2.pkl')
rf_leave_fluctuation_reg = joblib.load('./빅콘테스트/model/version2/rf_leave_fluctuation_reg_version2.pkl')
rf_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_version2.pkl')
xgb_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/xgb_amount_fluctuation_sub_reg_version2.pkl')
extree_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/extree_amount_fluctuation_sub_reg_version2.pkl')
lr_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/lr_amount_fluctuation_sub_reg_version2.pkl')

In [639]:
from score_function import score_function

w_list = [[0.1, 0.1, 0.1, 0.7], [0.1, 0.1, 0.2, 0.6], [0.1, 0.1, 0.3, 0.5], [0.1, 0.1, 0.4, 0.4],
          [0.1, 0.2, 0.2, 0.5], [0.1, 0.2, 0.3, 0.4], [0.2, 0.2, 0.2, 0.4], [0.2, 0.2, 0.3, 0.3]]

for i in range(0, len(w_list)):
    for j in range(0, 4):
        w1 = w_list[i][j % 4]
        w2 = w_list[i][(j + 1) % 4]
        w3 = w_list[i][(j + 2) % 4]
        w4 = w_list[i][(j + 3) % 4]
        
        test_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = rf_amount_fluctuation_sub_reg, 
                            model5 = xgb_amount_fluctuation_sub_reg, model6 = extree_amount_fluctuation_sub_reg,
                            model7 = lr_amount_fluctuation_sub_reg, data = X_test_fluctuation,
                            w1 = w1, w2 = w2, w3 = w3, w4 = w4)
        
        test_pred.to_csv('y_pred_ver2.csv', index = False)
        score = score_function('y_pred_ver2.csv', 'true.csv')
        print('w1 :', w1, 'w2 :', w2, 'w3 :', w3, 'w4 :', w4, '\t score :', score)

3640.767211032496
w1 : 0.1 w2 : 0.1 w3 : 0.1 w4 : 0.7 	 score : 3640.767211032496
3829.1210008854146
w1 : 0.1 w2 : 0.1 w3 : 0.7 w4 : 0.1 	 score : 3829.1210008854146
3463.651043618372
w1 : 0.1 w2 : 0.7 w3 : 0.1 w4 : 0.1 	 score : 3463.651043618372
3793.33419724041
w1 : 0.7 w2 : 0.1 w3 : 0.1 w4 : 0.1 	 score : 3793.33419724041
3685.6106830920303
w1 : 0.1 w2 : 0.1 w3 : 0.2 w4 : 0.6 	 score : 3685.6106830920303
3810.5799845224137
w1 : 0.1 w2 : 0.2 w3 : 0.6 w4 : 0.1 	 score : 3810.5799845224137
3575.118851309865
w1 : 0.2 w2 : 0.6 w3 : 0.1 w4 : 0.1 	 score : 3575.118851309865
3783.863933433345
w1 : 0.6 w2 : 0.1 w3 : 0.1 w4 : 0.2 	 score : 3783.863933433345
3722.3885260243915
w1 : 0.1 w2 : 0.1 w3 : 0.3 w4 : 0.5 	 score : 3722.3885260243915
3782.820789795731
w1 : 0.1 w2 : 0.3 w3 : 0.5 w4 : 0.1 	 score : 3782.820789795731
3669.8572272603033
w1 : 0.3 w2 : 0.5 w3 : 0.1 w4 : 0.1 	 score : 3669.8572272603033
3768.5415534086524
w1 : 0.5 w2 : 0.1 w3 : 0.1 w4 : 0.3 	 score : 3768.5415534086524
3757.0

In [641]:
from score_function import score_function

w_list = [[0.1, 0.1, 0.1, 1.2], [0.1, 0.1, 0.2, 1.1], [0.1, 0.1, 0.3, 1.0], [0.1, 0.1, 0.4, 0.9],
          [0.1, 0.1, 0.5, 0.8], [0.1, 0.1, 0.6, 0.7], [0.1, 0.2, 0.2, 1.0], [0.1, 0.2, 0.3, 0.9],
          [0.1, 0.2, 0.4, 0.8], [0.1, 0.2, 0.5, 0.7], [0.1, 0.2, 0.6, 0.6], [0.1, 0.3, 0.3, 0.8],
          [0.1, 0.3, 0.4, 0.7], [0.1, 0.3, 0.5, 0.6], [0.1, 0.4, 0.4, 0.6], [0.1, 0.4, 0.5, 0.5],
          [0.2, 0.2, 0.2, 0.9], [0.2, 0.2, 0.3, 0.8], [0.2, 0.2, 0.4, 0.7], [0.2, 0.2, 0.5, 0.6],
          [0.2, 0.3, 0.3, 0.7], [0.2, 0.3, 0.4, 0.6], [0.2, 0.3, 0.5, 0.5], [0.2, 0.4, 0.4, 0.5],
          [0.3, 0.3, 0.3, 0.6], [0.3, 0.3, 0.4, 0.5], [0.3, 0.4, 0.4, 0.4]]

for i in range(0, len(w_list)):
    for j in range(0, 4):
        w1 = w_list[i][j % 4]
        w2 = w_list[i][(j + 1) % 4]
        w3 = w_list[i][(j + 2) % 4]
        w4 = w_list[i][(j + 3) % 4]
        
        test_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = rf_amount_fluctuation_sub_reg, 
                            model5 = xgb_amount_fluctuation_sub_reg, model6 = extree_amount_fluctuation_sub_reg,
                            model7 = lr_amount_fluctuation_sub_reg, data = X_test_fluctuation,
                            w1 = w1, w2 = w2, w3 = w3, w4 = w4)
        
        test_pred.to_csv('y_pred_ver2.csv', index = False)
        score = score_function('y_pred_ver2.csv', 'true.csv')
        print('w1 :', w1, '   w2 :', w2, '   w3 :', w3, '   w4 :', w4, '\t score :', score)

4160.541067908602
w1 : 0.1    w2 : 0.1    w3 : 0.1    w4 : 1.2 	 score : 4160.541067908602
4421.273543027562
w1 : 0.1    w2 : 0.1    w3 : 1.2    w4 : 0.1 	 score : 4421.273543027562
4096.956601983207
w1 : 0.1    w2 : 1.2    w3 : 0.1    w4 : 0.1 	 score : 4096.956601983207
4405.254524529977
w1 : 1.2    w2 : 0.1    w3 : 0.1    w4 : 0.1 	 score : 4405.254524529977
4204.702383041608
w1 : 0.1    w2 : 0.1    w3 : 0.2    w4 : 1.1 	 score : 4204.702383041608
4426.422102547244
w1 : 0.1    w2 : 0.2    w3 : 1.1    w4 : 0.1 	 score : 4426.422102547244
4162.605129355092
w1 : 0.2    w2 : 1.1    w3 : 0.1    w4 : 0.1 	 score : 4162.605129355092
4412.021898253924
w1 : 1.1    w2 : 0.1    w3 : 0.1    w4 : 0.2 	 score : 4412.021898253924
4246.97582173398
w1 : 0.1    w2 : 0.1    w3 : 0.3    w4 : 1.0 	 score : 4246.97582173398
4428.815348938445
w1 : 0.1    w2 : 0.3    w3 : 1.0    w4 : 0.1 	 score : 4428.815348938445
4210.4999725033695
w1 : 0.3    w2 : 1.0    w3 : 0.1    w4 : 0.1 	 score : 4210.4999725033695

4413.3683364303715
w1 : 0.5    w2 : 0.2    w3 : 0.3    w4 : 0.5 	 score : 4413.3683364303715
4352.32424716088
w1 : 0.2    w2 : 0.4    w3 : 0.4    w4 : 0.5 	 score : 4352.32424716088
4427.738994000685
w1 : 0.4    w2 : 0.4    w3 : 0.5    w4 : 0.2 	 score : 4427.738994000685
4348.54836014545
w1 : 0.4    w2 : 0.5    w3 : 0.2    w4 : 0.4 	 score : 4348.54836014545
4428.622643078275
w1 : 0.5    w2 : 0.2    w3 : 0.4    w4 : 0.4 	 score : 4428.622643078275
4355.137524552851
w1 : 0.3    w2 : 0.3    w3 : 0.3    w4 : 0.6 	 score : 4355.137524552851
4430.182716960669
w1 : 0.3    w2 : 0.3    w3 : 0.6    w4 : 0.3 	 score : 4430.182716960669
4340.949775352894
w1 : 0.3    w2 : 0.6    w3 : 0.3    w4 : 0.3 	 score : 4340.949775352894
4429.983362361364
w1 : 0.6    w2 : 0.3    w3 : 0.3    w4 : 0.3 	 score : 4429.983362361364
4388.618539980445
w1 : 0.3    w2 : 0.3    w3 : 0.4    w4 : 0.5 	 score : 4388.618539980445
4412.045262402331
w1 : 0.3    w2 : 0.4    w3 : 0.5    w4 : 0.3 	 score : 4412.045262402331
4

In [657]:
from score_function import score_function

w_list = [[0.1, 0.1, 0.1, 1.7], [0.1, 0.1, 0.2, 1.6], [0.1, 0.1, 0.3, 1.5], [0.1, 0.1, 0.4, 1.4],
          [0.1, 0.1, 0.5, 1.3], [0.1, 0.1, 0.6, 1.2], [0.1, 0.1, 0.7, 1.1], [0.1, 0.1, 0.8, 1.0],
          [0.1, 0.1, 0.9, 0.9], [0.1, 0.2, 0.2, 1.5], [0.1, 0.2, 0.3, 1.4], [0.1, 0.2, 0.4, 1.3],
          [0.1, 0.2, 0.5, 1.2], [0.1, 0.2, 0.6, 1.1], [0.1, 0.2, 0.7, 1.0], [0.1, 0.2, 0.8, 0.9],
          [0.1, 0.3, 0.3, 1.3], [0.1, 0.3, 0.4, 1.2], [0.1, 0.3, 0.5, 1.1], [0.1, 0.3, 0.6, 1.0],
          [0.1, 0.3, 0.7, 0.9], [0.1, 0.3, 0.8, 0.8], [0.1, 0.4, 0.4, 1.1], [0.1, 0.4, 0.5, 1.0],
          [0.1, 0.4, 0.6, 0.9], [0.1, 0.4, 0.7, 0.8], [0.1, 0.5, 0.5, 0.9], [0.1, 0.5, 0.6, 0.8],
          [0.1, 0.5, 0.7, 0.7], [0.1, 0.6, 0.6, 0.7], [0.2, 0.2, 0.2, 1.4], [0.2, 0.2, 0.3, 1.3],
          [0.2, 0.2, 0.4, 1.2], [0.2, 0.2, 0.5, 1.1], [0.2, 0.2, 0.6, 1.0], [0.2, 0.2, 0.7, 0.9],
          [0.2, 0.2, 0.8, 0.8], [0.2, 0.3, 0.3, 1.2], [0.2, 0.3, 0.4, 1.1], [0.2, 0.3, 0.5, 1.0],
          [0.2, 0.3, 0.6, 0.9], [0.2, 0.3, 0.7, 0.8], [0.2, 0.4, 0.4, 1.0], [0.2, 0.4, 0.5, 0.9],
          [0.2, 0.4, 0.6, 0.8], [0.2, 0.4, 0.7, 0.7], [0.2, 0.5, 0.5, 0.8], [0.2, 0.5, 0.6, 0.7],
          [0.2, 0.6, 0.6, 0.6], [0.3, 0.3, 0.3, 1.1], [0.3, 0.3, 0.4, 1.0], [0.3, 0.3, 0.5, 0.9],
          [0.3, 0.3, 0.6, 0.8], [0.3, 0.3, 0.7, 0.7], [0.3, 0.4, 0.4, 0.9], [0.3, 0.4, 0.5, 0.8],
          [0.3, 0.4, 0.6, 0.7], [0.3, 0.5, 0.5, 0.7], [0.3, 0.5, 0.6, 0.6], [0.4, 0.4, 0.4, 0.8],
          [0.4, 0.4, 0.5, 0.7], [0.4, 0.4, 0.6, 0.6], [0.4, 0.5, 0.5, 0.6], [0.5, 0.5, 0.5, 0.5]]

for i in range(0, len(w_list)):
    for j in range(0, 4):
        w1 = w_list[i][j % 4]
        w2 = w_list[i][(j + 1) % 4]
        w3 = w_list[i][(j + 2) % 4]
        w4 = w_list[i][(j + 3) % 4]
        
        test_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = rf_amount_fluctuation_sub_reg, 
                            model5 = xgb_amount_fluctuation_sub_reg, model6 = extree_amount_fluctuation_sub_reg,
                            model7 = lr_amount_fluctuation_sub_reg, data = X_test_fluctuation,
                            w1 = w1, w2 = w2, w3 = w3, w4 = w4)
        
        test_pred.to_csv('y_pred_ver2.csv', index = False)
        score = score_function('y_pred_ver2.csv', 'true.csv')
        print('w1 :', w1, '   w2 :', w2, '   w3 :', w3, '   w4 :', w4, '\t score :', score)

4531.849821204991
w1 : 0.1    w2 : 0.1    w3 : 0.1    w4 : 1.7 	 score : 4531.849821204991
4753.79992343031
w1 : 0.1    w2 : 0.1    w3 : 1.7    w4 : 0.1 	 score : 4753.79992343031
4400.740323543372
w1 : 0.1    w2 : 1.7    w3 : 0.1    w4 : 0.1 	 score : 4400.740323543372
4765.658279963931
w1 : 1.7    w2 : 0.1    w3 : 0.1    w4 : 0.1 	 score : 4765.658279963931
4564.326025715472
w1 : 0.1    w2 : 0.1    w3 : 0.2    w4 : 1.6 	 score : 4564.326025715472
4760.021155919261
w1 : 0.1    w2 : 0.2    w3 : 1.6    w4 : 0.1 	 score : 4760.021155919261
4452.133081708799
w1 : 0.2    w2 : 1.6    w3 : 0.1    w4 : 0.1 	 score : 4452.133081708799
4775.981031960594
w1 : 1.6    w2 : 0.1    w3 : 0.1    w4 : 0.2 	 score : 4775.981031960594
4587.318322921732
w1 : 0.1    w2 : 0.1    w3 : 0.3    w4 : 1.5 	 score : 4587.318322921732
4761.530507379546
w1 : 0.1    w2 : 0.3    w3 : 1.5    w4 : 0.1 	 score : 4761.530507379546
4503.473333574199
w1 : 0.3    w2 : 1.5    w3 : 0.1    w4 : 0.1 	 score : 4503.473333574199
4

4780.703432298006
w1 : 1.1    w2 : 0.1    w3 : 0.4    w4 : 0.4 	 score : 4780.703432298006
4687.275190631407
w1 : 0.1    w2 : 0.4    w3 : 0.5    w4 : 1.0 	 score : 4687.275190631407
4768.917759404262
w1 : 0.4    w2 : 0.5    w3 : 1.0    w4 : 0.1 	 score : 4768.917759404262
4680.730889013387
w1 : 0.5    w2 : 1.0    w3 : 0.1    w4 : 0.4 	 score : 4680.730889013387
4771.9707568053645
w1 : 1.0    w2 : 0.1    w3 : 0.4    w4 : 0.5 	 score : 4771.9707568053645
4702.47962910192
w1 : 0.1    w2 : 0.4    w3 : 0.6    w4 : 0.9 	 score : 4702.47962910192
4761.341231685654
w1 : 0.4    w2 : 0.6    w3 : 0.9    w4 : 0.1 	 score : 4761.341231685654
4714.455000520872
w1 : 0.6    w2 : 0.9    w3 : 0.1    w4 : 0.4 	 score : 4714.455000520872
4762.462713185534
w1 : 0.9    w2 : 0.1    w3 : 0.4    w4 : 0.6 	 score : 4762.462713185534
4714.974240415777
w1 : 0.1    w2 : 0.4    w3 : 0.7    w4 : 0.8 	 score : 4714.974240415777
4751.8493637903175
w1 : 0.4    w2 : 0.7    w3 : 0.8    w4 : 0.1 	 score : 4751.84936379031

4749.910957091748
w1 : 0.4    w2 : 0.7    w3 : 0.7    w4 : 0.2 	 score : 4749.910957091748
4742.421772522048
w1 : 0.7    w2 : 0.7    w3 : 0.2    w4 : 0.4 	 score : 4742.421772522048
4748.645982393008
w1 : 0.7    w2 : 0.2    w3 : 0.4    w4 : 0.7 	 score : 4748.645982393008
4714.862219269482
w1 : 0.2    w2 : 0.5    w3 : 0.5    w4 : 0.8 	 score : 4714.862219269482
4770.01246335398
w1 : 0.5    w2 : 0.5    w3 : 0.8    w4 : 0.2 	 score : 4770.01246335398
4722.401534820453
w1 : 0.5    w2 : 0.8    w3 : 0.2    w4 : 0.5 	 score : 4722.401534820453
4767.785809646316
w1 : 0.8    w2 : 0.2    w3 : 0.5    w4 : 0.5 	 score : 4767.785809646316
4726.519693729717
w1 : 0.2    w2 : 0.5    w3 : 0.6    w4 : 0.7 	 score : 4726.519693729717
4762.19340235565
w1 : 0.5    w2 : 0.6    w3 : 0.7    w4 : 0.2 	 score : 4762.19340235565
4734.458119289754
w1 : 0.6    w2 : 0.7    w3 : 0.2    w4 : 0.5 	 score : 4734.458119289754
4758.342842909536
w1 : 0.7    w2 : 0.2    w3 : 0.5    w4 : 0.6 	 score : 4758.342842909536
472

In [658]:
test1_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = rf_amount_fluctuation_sub_reg, 
                            model5 = xgb_amount_fluctuation_sub_reg, model6 = extree_amount_fluctuation_sub_reg,
                            model7 = lr_amount_fluctuation_sub_reg, data = X_test1_fluctuation,
                            w1 = 1.3, w2 = 0.1, w3 = 0.3, w4 = 0.3)

In [659]:
test2_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = rf_amount_fluctuation_sub_reg, 
                            model5 = xgb_amount_fluctuation_sub_reg, model6 = extree_amount_fluctuation_sub_reg,
                            model7 = lr_amount_fluctuation_sub_reg, data = X_test2_fluctuation,
                            w1 = 1.3, w2 = 0.1, w3 = 0.3, w4 = 0.3)

In [660]:
print((test1_pred['survival_time'] > 64).sum())
print((test2_pred['survival_time'] > 64).sum())
print((test1_pred['survival_time'] <= 0).sum())
print((test2_pred['survival_time'] <= 0).sum())

0
0
0
0


In [661]:
print((test1_pred['amount_spent'] < 0).sum())
print((test2_pred['amount_spent'] < 0).sum())

0
0


In [662]:
print(test_pred['amount_spent'].sum())
print(test1_pred['amount_spent'].sum())
print(test2_pred['amount_spent'].sum())

3946.30190619997
11250.677488128511
14704.461428453784


In [663]:
test1_pred.to_csv('test1_predict.csv', index = False)
test2_pred.to_csv('test2_predict.csv', index = False)

In [671]:
def predict(model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, data,
            w1, w2, w3, w4, w5, w6, w7, w8):
    '''
    model1 : 생존 여부 예측(classifier)
    model2 : 결제 여부 예측(classifier)
    model3~6 : 추가 생존 기간 예측(regressor)
    model7~10 : 일 평균 결제 금액 예측(regressor)
    data : test data
    '''
    
    pred1 = model1.predict(data.iloc[:, 1:])
    pred2 = model2.predict(data.iloc[:, 1:])
    pred3 = ((w1 * model3.predict(data.iloc[:, 1:])) + (w2 * model4.predict(data.iloc[:, 1:]))
            + (w3 * model5.predict(data.iloc[:, 1:])) + (w4 * model6.predict(data.iloc[:, 1:])))
    pred4 = ((w5 * model7.predict(data.iloc[:, 1:])) + (w6 * model8.predict(data.iloc[:, 1:]))
            + (w7 * model9.predict(data.iloc[:, 1:])) + (w8 * model10.predict(data.iloc[:, 1:])))
    
    pred3[pred1 == 0] = 64
    pred4[pred2 == 0] = 0
    pred3[pred3 <= 1] = 1
    pred3[pred3 >= 64] = 64
    pred4[pred4 <= 0] = 0
    
    array = np.concatenate([data.iloc[:, 0].values.reshape(-1, 1), pred3.reshape(-1, 1), pred4.reshape(-1, 1)], axis = 1)
    df = pd.DataFrame(array)
    df.columns = ['acc_id', 'survival_time', 'amount_spent']
    df['acc_id'] = df['acc_id'].astype('int32')
    df['survival_time'] = round(df['survival_time']).astype('int32')
    
    return df

In [4]:
X_test_fluctuation = pd.read_csv('X_test_ver2_fluctuation.csv')
X_test1_fluctuation = pd.read_csv('X_test1_ver2_fluctuation.csv')
X_test2_fluctuation = pd.read_csv('X_test2_ver2_fluctuation.csv')

In [7]:
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier

voting_leave_fluctuation_clf_2 = joblib.load('./빅콘테스트/model/version2/voting_leave_fluctuation_clf_2_version2.pkl')
voting_amount_fluctuation_clf_2 = joblib.load('./빅콘테스트/model/version2/voting_amount_fluctuation_clf_2_version2.pkl')
rf_leave_fluctuation_reg = joblib.load('./빅콘테스트/model/version2/rf_leave_fluctuation_reg_version2.pkl')
xgb_leave_fluctuation_reg = joblib.load('./빅콘테스트/model/version2/xgb_leave_fluctuation_reg_version2.pkl')
extree_leave_fluctuation_reg = joblib.load('./빅콘테스트/model/version2/extree_leave_fluctuation_reg_version2.pkl')
lr_leave_fluctuation_reg = joblib.load('./빅콘테스트/model/version2/lr_leave_fluctuation_reg_version2.pkl')
rf_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_version2.pkl')
xgb_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/xgb_amount_fluctuation_sub_reg_version2.pkl')
extree_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/extree_amount_fluctuation_sub_reg_version2.pkl')
lr_amount_fluctuation_sub_reg = joblib.load('./빅콘테스트/model/version2/lr_amount_fluctuation_sub_reg_version2.pkl')

ModuleNotFoundError: No module named 'sklearn.ensemble.voting'

In [675]:
from score_function import score_function

w_list = [[0.1, 0.1, 0.1, 0.7], [0.1, 0.1, 0.15, 0.65], [0.1, 0.1, 0.2, 0.6], [0.1, 0.1, 0.25, 0.55],
          [0.1, 0.1, 0.3, 0.5], [0.1, 0.1, 0.35, 0.45], [0.1, 0.1, 0.4, 0.4], [0.1, 0.15, 0.15, 0.6],
          [0.1, 0.15, 0.2, 0.55], [0.1, 0.15, 0.25, 0.5], [0.1, 0.15, 0.3, 0.45], [0.1, 0.15, 0.35, 0.4],
          [0.1, 0.2, 0.2, 0.5], [0.1, 0.2, 0.25, 0.45], [0.1, 0.2, 0.3, 0.4], [0.1, 0.2, 0.35, 0.35],
          [0.1, 0.25, 0.25, 0.4], [0.1, 0.25, 0.3, 0.35], [0.1, 0.3, 0.3, 0.3], [0.15, 0.15, 0.15, 0.55],
          [0.15, 0.15, 0.2, 0.5], [0.15, 0.15, 0.25, 0.45], [0.15, 0.15, 0.3, 0.4], [0.15, 0.15, 0.35, 0.35],
          [0.15, 0.2, 0.2, 0.45], [0.15, 0.2, 0.25, 0.4], [0.15, 0.2, 0.3, 0.35], [0.15, 0.25, 0.25, 0.35],
          [0.15, 0.25, 0.3, 0.3], [0.2, 0.2, 0.2, 0.4], [0.2, 0.2, 0.25, 0.35], [0.2, 0.2, 0.3, 0.3],
          [0.25, 0.25, 0.25, 0.25]]

for i in range(0, len(w_list)):
    for j in range(0, 4):
        w1 = w_list[i][j % 4]
        w2 = w_list[i][(j + 1) % 4]
        w3 = w_list[i][(j + 2) % 4]
        w4 = w_list[i][(j + 3) % 4]
        
        test_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = xgb_leave_fluctuation_reg,
                            model5 = extree_leave_fluctuation_reg, model6 = lr_leave_fluctuation_reg,
                            model7 = rf_amount_fluctuation_sub_reg, model8 = xgb_amount_fluctuation_sub_reg,
                            model9 = extree_amount_fluctuation_sub_reg, model10 = lr_amount_fluctuation_sub_reg,
                            data = X_test_fluctuation,
                            w1 = w1, w2 = w2, w3 = w3, w4 = w4, w5 = 1.3, w6 = 0.1, w7 = 0.3, w8 = 0.3)
        
        test_pred.to_csv('y_pred_ver2.csv', index = False)
        score = score_function('y_pred_ver2.csv', 'true.csv')
        print('w1 :', w1, '   w2 :', w2, '   w3 :', w3, '   w4 :', w4, '\t score :', score)

4390.43529592148
w1 : 0.1    w2 : 0.1    w3 : 0.1    w4 : 0.7 	 score : 4390.43529592148
4747.324262672277
w1 : 0.1    w2 : 0.1    w3 : 0.7    w4 : 0.1 	 score : 4747.324262672277
4803.956817448573
w1 : 0.1    w2 : 0.7    w3 : 0.1    w4 : 0.1 	 score : 4803.956817448573
4779.592161150238
w1 : 0.7    w2 : 0.1    w3 : 0.1    w4 : 0.1 	 score : 4779.592161150238
4441.9229636899245
w1 : 0.1    w2 : 0.1    w3 : 0.15    w4 : 0.65 	 score : 4441.9229636899245
4756.811097509668
w1 : 0.1    w2 : 0.15    w3 : 0.65    w4 : 0.1 	 score : 4756.811097509668
4805.59298390803
w1 : 0.15    w2 : 0.65    w3 : 0.1    w4 : 0.1 	 score : 4805.59298390803
4765.540679497262
w1 : 0.65    w2 : 0.1    w3 : 0.1    w4 : 0.15 	 score : 4765.540679497262
4483.780372892104
w1 : 0.1    w2 : 0.1    w3 : 0.2    w4 : 0.6 	 score : 4483.780372892104
4761.148522231858
w1 : 0.1    w2 : 0.2    w3 : 0.6    w4 : 0.1 	 score : 4761.148522231858
4801.03627184851
w1 : 0.2    w2 : 0.6    w3 : 0.1    w4 : 0.1 	 score : 4801.0362718

4764.317947216658
w1 : 0.15    w2 : 0.3    w3 : 0.4    w4 : 0.15 	 score : 4764.317947216658
4774.302500636768
w1 : 0.3    w2 : 0.4    w3 : 0.15    w4 : 0.15 	 score : 4774.302500636768
4694.744262503414
w1 : 0.4    w2 : 0.15    w3 : 0.15    w4 : 0.3 	 score : 4694.744262503414
4662.512577139302
w1 : 0.15    w2 : 0.15    w3 : 0.35    w4 : 0.35 	 score : 4662.512577139302
4766.3609438938065
w1 : 0.15    w2 : 0.35    w3 : 0.35    w4 : 0.15 	 score : 4766.3609438938065
4772.517748063041
w1 : 0.35    w2 : 0.35    w3 : 0.15    w4 : 0.15 	 score : 4772.517748063041
4675.794028836311
w1 : 0.35    w2 : 0.15    w3 : 0.15    w4 : 0.35 	 score : 4675.794028836311
4606.650707812594
w1 : 0.15    w2 : 0.2    w3 : 0.2    w4 : 0.45 	 score : 4606.650707812594
4755.018874451832
w1 : 0.2    w2 : 0.2    w3 : 0.45    w4 : 0.15 	 score : 4755.018874451832
4760.278048473725
w1 : 0.2    w2 : 0.45    w3 : 0.15    w4 : 0.2 	 score : 4760.278048473725
4745.5287259607085
w1 : 0.45    w2 : 0.15    w3 : 0.2    w4 

In [703]:
test_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = xgb_leave_fluctuation_reg,
                            model5 = extree_leave_fluctuation_reg, model6 = lr_leave_fluctuation_reg,
                            model7 = rf_amount_fluctuation_sub_reg, model8 = xgb_amount_fluctuation_sub_reg,
                            model9 = extree_amount_fluctuation_sub_reg, model10 = lr_amount_fluctuation_sub_reg,
                            data = X_test_fluctuation,
                            w1 = 0.15, w2 = 0.65, w3 = 0.1, w4 = 0.1, w5 = 1.4 * 1.5, w6 = 0.2 * 1.5, w7 = 0.2 * 1.5, w8 = 0.2 * 1.5) 

In [704]:
test_pred.to_csv('y_pred_ver2.csv', index = False)
score = score_function('y_pred_ver2.csv', 'true.csv')

5232.479803251927


In [705]:
test1_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = xgb_leave_fluctuation_reg,
                            model5 = extree_leave_fluctuation_reg, model6 = lr_leave_fluctuation_reg,
                            model7 = rf_amount_fluctuation_sub_reg, model8 = xgb_amount_fluctuation_sub_reg,
                            model9 = extree_amount_fluctuation_sub_reg, model10 = lr_amount_fluctuation_sub_reg,
                            data = X_test1_fluctuation,
                            w1 = 0.15, w2 = 0.65, w3 = 0.1, w4 = 0.1, w5 = 1.4 * 1.5, w6 = 0.2 * 1.5, w7 = 0.2 * 1.5, w8 = 0.2 * 1.5)

In [706]:
test2_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = xgb_leave_fluctuation_reg,
                            model5 = extree_leave_fluctuation_reg, model6 = lr_leave_fluctuation_reg,
                            model7 = rf_amount_fluctuation_sub_reg, model8 = xgb_amount_fluctuation_sub_reg,
                            model9 = extree_amount_fluctuation_sub_reg, model10 = lr_amount_fluctuation_sub_reg,
                            data = X_test2_fluctuation,
                            w1 = 0.15, w2 = 0.65, w3 = 0.1, w4 = 0.1, w5 = 1.4 * 1.5, w6 = 0.2 * 1.5, w7 = 0.2 * 1.5, w8 = 0.2 * 1.5)

In [707]:
print((test1_pred['survival_time'] > 64).sum())
print((test2_pred['survival_time'] > 64).sum())
print((test1_pred['survival_time'] <= 0).sum())
print((test2_pred['survival_time'] <= 0).sum())

0
0
0
0


In [708]:
print((test1_pred['amount_spent'] < 0).sum())
print((test2_pred['amount_spent'] < 0).sum())

0
0


In [709]:
print(test_pred['amount_spent'].sum())
print(test1_pred['amount_spent'].sum())
print(test2_pred['amount_spent'].sum())

5969.78460560885
17298.998635161137
22546.065712006937


In [710]:
test1_pred.to_csv('test1_predict.csv', index = False)
test2_pred.to_csv('test2_predict.csv', index = False)

# bayesian optimization

In [711]:
def black_box_function(w1, w2, w3, w4, w5, w6, w7, w8):
    test_pred = predict(model1 = voting_leave_fluctuation_clf_2, model2 = voting_amount_fluctuation_clf_2, 
                            model3 = rf_leave_fluctuation_reg, model4 = xgb_leave_fluctuation_reg,
                            model5 = extree_leave_fluctuation_reg, model6 = lr_leave_fluctuation_reg,
                            model7 = rf_amount_fluctuation_sub_reg, model8 = xgb_amount_fluctuation_sub_reg,
                            model9 = extree_amount_fluctuation_sub_reg, model10 = lr_amount_fluctuation_sub_reg,
                            data = X_test_fluctuation,
                            w1 = w1, w2 = w2, w3 = w3, w4 = w4, w5 = w5, w6 = w6, w7 = w7, w8 = w8)
    
    test_pred.to_csv('y_pred_ver2.csv', index = False)
    score = score_function('y_pred_ver2.csv', 'true.csv')
    return score

In [719]:
from bayes_opt import BayesianOptimization

pbounds = {'w1' : (0, 1), 'w2' : (0, 1), 'w3' : (0, 1), 'w4' : (0, 1),
          'w5' : (0, 3), 'w6' : (0, 3), 'w7' : (0, 3), 'w8' : (0, 3)}

optimizer = BayesianOptimization(
    f = black_box_function,
    pbounds = pbounds,
    random_state = 42,
)

ModuleNotFoundError: No module named 'sklearn.gaussian_process'

#### 앙상블

In [613]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

for i in range(1, 16):
    rf_amount_fluctuation_sub_reg = RandomForestRegressor(random_state = 10 * i, max_features = 9, n_estimators = 300)
    rf_amount_fluctuation_sub_reg.fit(train_fluctuation_sub.iloc[:, 1:-4], train_fluctuation_sub['amount_spent'])
    model_path = './빅콘테스트/model/version2/rf_amount_fluctuation_sub_reg_' + str(i) + '_version2.pkl'
    joblib.dump(rf_amount_fluctuation_sub_reg, model_path)